In [ ]:
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
import torch.nn as nn
from torchvision import datasets,transforms,models
from torchvision.transforms.transforms import ToPILImage
import torch.optim as optim
from PIL import ImageFile,Image
from tqdm import tqdm
from google.colab.patches import cv2_imshow

**Checking for device**

In [ ]:
#Checking for device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
resize = (224,224)
batch_size = 32
num_epochs = 100

mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

In [ ]:
data_dir = '/content/drive/MyDrive/Colab Notebooks/COVID-LVTN/split_data'
len(glob.glob(data_dir+'/**/**/*.jpg'))

9208

**Image Transform**

In [ ]:
#image transform
img_transform = {
    'train':transforms.Compose([
                                transforms.Resize(resize),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomRotation(90),
                                transforms.ToTensor(),
                                transforms.Normalize(mean,std)
    ]),
    'val':transforms.Compose([
                              transforms.Resize(resize),
                              transforms.ToTensor(),
                              transforms.Normalize(mean,std)
    ]),
    'test':transforms.Compose([
                               transforms.Resize(resize),
                               transforms.ToTensor(),
                               transforms.Normalize(mean,std)
    ])
}

**Datasets**

In [ ]:
train_path = '/content/drive/MyDrive/Colab Notebooks/COVID-LVTN/split_data/train'
val_path = '/content/drive/MyDrive/Colab Notebooks/COVID-LVTN/split_data/valid'
test_path = '/content/drive/MyDrive/Colab Notebooks/COVID-LVTN/split_data/test'

In [ ]:
#dataset
train_file = datasets.ImageFolder(train_path,transform=img_transform['train'])
val_file = datasets.ImageFolder(val_path,transform=img_transform['val'])
test_file = datasets.ImageFolder(test_path,transform=img_transform['test'])

In [ ]:
train_count = len(glob.glob(train_path+'/**/*.jpg'))
val_count = len(glob.glob(val_path+'/**/*.jpg'))
test_count = len(glob.glob(test_path+'/**/*.jpg'))
print(train_count,val_count,test_count)

5526 1841 1841


**Dataloader**

In [ ]:
#dataloader
loader = {
    'train':torch.utils.data.DataLoader(train_file,batch_size,shuffle=True),
    'val':torch.utils.data.DataLoader(val_file,batch_size,shuffle=False),
    'test':torch.utils.data.DataLoader(test_file,batch_size,shuffle=False)
}

**Model**

In [ ]:
#use pretrained model
model = models.efficientnet_b0(pretrained=True)
for param in model.parameters():
      param.requires_grad = True
model.classifier[1] = nn.Linear(1280,4)
checkpoint = torch.load('/content/drive/MyDrive/Colab Notebooks/COVID-LVTN/checkpoint_effb0_3.pt',map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device);

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth


  0%|          | 0.00/20.5M [00:00<?, ?B/s]

**Loss**

In [ ]:
#loss
criterior = nn.CrossEntropyLoss()

**Optimizer**

In [ ]:
#optimizer
optimizer = optim.Adam(model.parameters(),lr=0.001)

**Train model**

In [ ]:
#Load checkpoint
state_dict = checkpoint['model_state_dict']
train_accu = checkpoint['train_accu']
train_losses = checkpoint['train_losses']
val_accu = checkpoint['val_accu']
val_losses = checkpoint['val_losses']
best_accuracy = checkpoint['best_accuracy']
epoch = checkpoint['epoch']

In [ ]:
# Train model
# best_accuracy = 0.0
# train_accu = []
# train_losses = []
# val_accu = []
# val_losses = []
# epoch = 0

for epochh in range(epoch+1,num_epochs+1):
  print("Epoch {}/{}".format(epochh,num_epochs))

  #Evaluation and training on training dataset
  model.train()
  train_accuracy = 0.0
  train_loss = 0.0

  for inputs,labels in tqdm(loader['train']):
    inputs = inputs.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()

    outputs=model(inputs)
    loss = criterior(outputs,labels)
    loss.backward()
    optimizer.step()

    train_loss += loss.cpu().data*inputs.size(0)
    _,prediction = torch.max(outputs.data,1)

    train_accuracy += torch.sum(prediction==labels.data)

  train_accuracy = train_accuracy/train_count
  train_loss = train_loss/train_count

  train_accu.append(train_accuracy)
  train_losses.append(train_loss) 

  # Evaluation on validation dataset
  model.eval()
  val_accuracy = 0.0
  val_loss = 0.0

  with torch.no_grad():
    for inputs,labels in tqdm(loader['val']):
      inputs = inputs.to(device)
      labels = labels.to(device)

      outputs = model(inputs)
      loss = criterior(outputs,labels)

      val_loss += loss.cpu().data*inputs.size(0)
      _,prediction = torch.max(outputs.data,1)

      val_accuracy += torch.sum(prediction==labels.data)

  val_accuracy = val_accuracy/val_count
  val_loss = val_loss/val_count

  val_accu.append(val_accuracy)
  val_losses.append(val_loss)

  print("Train Loss: {:.4f}, Train Accuracy: {:.4f}, Val Loss: {:.4f}, Val Accuracy: {:.4f}".format(train_loss,train_accuracy,val_loss,val_accuracy))

  if val_accuracy > best_accuracy:
    state_dict = model.state_dict()
    best_accuracy = val_accuracy

  #Save model
  torch.save({
      'epoch':epochh,
      'best_accuracy':best_accuracy,
      'train_accu':train_accu,
      'train_losses':train_losses,
      'val_accu':val_accu,
      'val_losses':val_losses,
      'model_state_dict':state_dict
  },'/content/drive/MyDrive/Colab Notebooks/COVID-LVTN/checkpoint_effb0_3.pt')

Epoch 85/100


100%|██████████| 58/58 [10:34<00:00, 10.94s/it]


Train Loss: 0.1268, Train Accuracy: 0.9544, Val Loss: 0.5374, Val Accuracy: 0.8664
Epoch 86/100


100%|██████████| 58/58 [00:55<00:00,  1.04it/s]


Train Loss: 0.1139, Train Accuracy: 0.9631, Val Loss: 0.6985, Val Accuracy: 0.8224
Epoch 87/100


...

Train Loss: 0.0894, Train Accuracy: 0.9682, Val Loss: 0.6173, Val Accuracy: 0.8626
Epoch 99/100


100%|██████████| 58/58 [00:55<00:00,  1.05it/s]


Train Loss: 0.0785, Train Accuracy: 0.9696, Val Loss: 0.7000, Val Accuracy: 0.8718
Epoch 100/100


100%|██████████| 58/58 [00:54<00:00,  1.06it/s]

Train Loss: 0.0736, Train Accuracy: 0.9720, Val Loss: 0.6087, Val Accuracy: 0.8631
